In [28]:
import pandas as pd
from pathlib import Path
import csv

# Load one of the WUG datasets. Run only one of the cells!

In [47]:
dfs = []
for p in Path('../').glob('RuDSIfixed/data/*/uses.csv'):
    df = pd.read_csv(p, quoting=csv.QUOTE_MINIMAL, sep='\t')
    original_columns = df.columns
    df['path'] = p
#     ds = p.parent.parent.parent.name
#     df['part'] = ds
#     df['lang'] = 'ru'
    dfs.append(df)
df = pd.concat(dfs, ignore_index=True)
df['positions'] = df['indexes_target_token'].str.split(':').apply(lambda l: list(map(int, l)))
df.head()

,lemma,pos,date,grouping,identifier,description,context,indexes_target_token,indexes_target_sentence,identifier_system,project,lang,user,path,positions
0,бог,NN,example,22,22_бог_23785,NaN,"Какъ сынъ бога, медвѣдь является представителемъ правды за землѣ.",10:14,0:65,27726,sem_cluster,ru,ek.gavrishina,../RuDSIfixed/data/бог/uses.csv,"[10, 14]"
1,бог,NN,example,22,22_бог_63941,NaN,"-- Да знаю я, слава богу.",20:24,0:25,27739,sem_cluster,ru,ek.gavrishina,../RuDSIfixed/data/бог/uses.csv,"[20, 24]"
2,бог,NN,example,22,22_бог_16754,NaN,"«Забыть?-- забвенья не дал бог: да он и не взял бы забвенья!»,«...Спастись от думы неизбежной и незабвенное забыть!»,«...И лучших дней воспоминанья пред ним теснилися толпой»,«В душе моей, с начала мира, твой образ был запечатлен...».",27:30,0:234,27720,sem_cluster,ru,ek.gavrishina,../RuDSIfixed/data/бог/uses.csv,"[27, 30]"
3,бог,NN,example,22,22_бог_6152,NaN,Тамань-- богом забытый далекий куток на побережье.,9:14,0:50,27741,sem_cluster,ru,ek.gavrishina,../RuDSIfixed/data/бог/uses.csv,"[9, 14]"
4,бог,NN,example,22,22_бог_42594,NaN,"Если не примут серьезных мер, скоро не только исчезнет вся деревянная Русь, но и каменные здания, не угодившие гостинодворскому вкусу малообразованных заправил, будут перекромсаны и разделаны, вроде той чудесной церкви древнего монастыря в Ростове Ярославском, в котором настоятель счистил фрески и покрыл его розовым стюком под мрамор,-суди его бог!",346:349,0:350,27729,sem_cluster,ru,ek.gavrishina,../RuDSIfixed/data/бог/uses.csv,"[346, 349]"


In [48]:
len(df)

840

# Filtering / calculating correct positions of the target word.

In [49]:
fdf = pd.read_csv('../fixes/RuDSI_forms_manual_fixed.tsv', sep='\t')
fdf

,words,forms
0,бог,бог бога богу Бог богом Боже богов
1,время,время времени времена Время времен временем
2,год,года годы год году годъ годах лет
3,голова,голову головой головы Голова голове голова голов
4,город,города город городе городов городъ городы Город городах
5,государство,государства государств государство государству государстве государствами государствам государствах государством
6,дело,дело делу дела деле делами Дело делах дел
7,день,день дня днях дни днем дне дней днями
8,друг,друг другом друзьями друзей друзья друга другу Друзья друзьям
9,жена,жена жену жене жены Жена женою женой жен женах


In [50]:
# fdf = pd.read_csv('../word_forms/ru_forms.tsv', sep='\t')
# fdf.forms = fdf.forms.str.strip()  # fixing the issue with the obsolete \n in the end of each line
# fdf = fdf.merge(pd.read_csv('./ru_forms_manual.tsv', sep='\t'), on='words', how='left')
# fdf['forms'] = fdf.forms_x + (' ' + fdf.forms_y).where(~fdf.forms_y.isnull(), '')  # concatenate forms from 2 files
# fdf

In [51]:
import re
fdf['forms'] = fdf.forms.apply(lambda s: r'(^|\W)(' + r'|'.join(re.escape(f) for f in s.split()) + r')(\W|$)' )
fdf[['words','forms']]

,words,forms
0,бог,(^|\W)(бог|бога|богу|Бог|богом|Боже|богов)(\W|$)
1,время,(^|\W)(время|времени|времена|Время|времен|временем)(\W|$)
2,год,(^|\W)(года|годы|год|году|годъ|годах|лет)(\W|$)
3,голова,(^|\W)(голову|головой|головы|Голова|голове|голова|голов)(\W|$)
4,город,(^|\W)(города|город|городе|городов|городъ|городы|Город|городах)(\W|$)
5,государство,(^|\W)(государства|государств|государство|государству|государстве|государствами|государствам|государствах|государством)(\W|$)
6,дело,(^|\W)(дело|делу|дела|деле|делами|Дело|делах|дел)(\W|$)
7,день,(^|\W)(день|дня|днях|дни|днем|дне|дней|днями)(\W|$)
8,друг,(^|\W)(друг|другом|друзьями|друзей|друзья|друга|другу|Друзья|друзьям)(\W|$)
9,жена,(^|\W)(жена|жену|жене|жены|Жена|женою|женой|жен|женах)(\W|$)


In [52]:
import re
patterns = {l:re.compile(s, flags=re.IGNORECASE) for l,s in fdf.set_index('words').forms.to_dict().items()}
print(next(iter(patterns.values())))
print(patterns['человек'])
m = patterns['человек'].search('Сначала присылали ему хлебца, круп, того-другого, да как узнали, что он все чужим людям раздает, так и полно!')
print(m.groups()[1])
print(m.span(2))

re.compile('(^|\\W)(бог|бога|богу|Бог|богом|Боже|богов)(\\W|$)', re.IGNORECASE)
re.compile('(^|\\W)(человек|человека|Человек|люди|людей|людьми|людях|людям)(\\W|$)', re.IGNORECASE)
людям
(82, 87)


In [53]:
mdf = df.apply(lambda r: patterns[r.lemma].search(r.context), axis=1)  # search for forms of the declared lemma
df['positions_fixed'] = mdf.apply(lambda m: None if m is None else list(m.span(2)))
print('The proportion of uses with no forms of the declared lemma found:', mdf.isnull().mean())
print('The number of uses with no forms of the declared lemma found:', df[mdf.isnull()].lemma.value_counts(), sep='\n')

The proportion of uses with no forms of the declared lemma found: 0.007142857142857143
The number of uses with no forms of the declared lemma found:
город    4
год      2
Name: lemma, dtype: int64


In [54]:
pd.set_option('max_colwidth',500)
df[mdf.isnull()][['lemma','context']].sort_values(by='lemma')

,lemma,context
706,год,"Так не потому ли, г. Шебуев, интеллигенция пока ударилась в книжную жизнь?"
716,год,"Я безпрестанно обращался по этому поводу къ г. Михайловскому, но онъ относился отрицательно къ_моему предложенію сдѣлать запросъ и попытаться установить нѣкоторое единство въ этомъ случаѣ."
525,город,Главный офис компании находится в г. Новый Уренгой(Ямало-Ненецкий автономный округ).
540,город,"У Гоголя, например, жизнь, как он сам выражается, действительно_щеголяет_по-будничному; она тем сильнее и жалче, чем сильнее щеголяет собою; у г."
554,город,"И если б г. Майков был вполне искренен, то он, конечно, не колеблясь, стал бы на сторону«папеньки», ибо, в сущности, здесь самые противоречия устраняются очень легко, ибо здесь похвальные качества до такой степени примиряются с качествами непохвальными, что едва ли даже отыщется в целом мире то тонкое перо, которое в состоянии провести между ними разделяющую черту."
558,город,Последние книжки необыкновенно популярного в 40-е годы французского романиста были отобраны у Достоевского во время обыска 1847 г.


In [55]:
print('Before drop: ', len(df))
df.drop(df[df.positions_fixed.isnull()].index, inplace=True)
print('After drop: ', len(df))

Before drop:  840
After drop:  834


# Loding datasets (continued)

In [3]:
dfs = []
for p in Path('./').glob('nor_dia_change/*/data/*/uses.csv'):
    df = pd.read_csv(p, quoting=csv.QUOTE_MINIMAL, sep='\t')
    ds = p.parent.parent.parent.name
    df['part'] = ds.replace('subset','nordiachange')
    df['lang'] = 'no'
    dfs.append(df)
df = pd.concat(dfs, ignore_index=True)
df.drop(df.index[df.indexes_target_token=='0:0'], inplace=True)  # TODO: fix in the dataset!
df['positions'] = df['indexes_target_token'].str.split(':').apply(lambda l: list(map(int, l)))
df['positions'] = df['positions'].apply(lambda l: [l[0], l[1]+1])  # TODO: fix in the dataset!
df.head(3)

,lemma,pos,date,grouping,identifier,description,context,indexes_target_token,indexes_target_sentence,identifier_system,project,lang,user,part,positions
0,driftsregnskap,NN,1980,1980-1990,1980-1990_driftsregnskap_71,NaN,En slik «akseptabel» balanse vil i høyprisscen...,68:82,0:216,15225,nor_dia_change_subset2,no,andrey,nordiachange2,"[68, 83]"
1,driftsregnskap,NN,1980,1980-1990,1980-1990_driftsregnskap_48,NaN,"Driftsregnskapet overfor utlandet, overskudd/u...",0:16,0:376,15230,nor_dia_change_subset2,no,andrey,nordiachange2,"[0, 17]"
2,driftsregnskap,NN,2012,2012-2019,2012-2019_driftsregnskap_20,NaN,I Rogaland er det flere som har høyere over ti...,381:397,0:727,15224,nor_dia_change_subset2,no,andrey,nordiachange2,"[381, 398]"


In [2]:
dfs = []
for p in Path('./').glob('dwug_de/data/*/uses.csv'):
    df = pd.read_csv(p, quoting=csv.QUOTE_NONE, sep='\t')
    ds = p.parent.parent.parent.name
    df['part'] = ds
    df['lang'] = ds.split('_')[1]
    dfs.append(df)
df = pd.concat(dfs, ignore_index=True)
df['positions'] = df['indexes_target_token'].str.split(':').apply(lambda l: list(map(int, l)))
df.head(3)

,lemma,pos,date,grouping,identifier,description,context,indexes_target_token,indexes_target_sentence,context_tokenized,indexes_target_token_tokenized,indexes_target_sentence_tokenized,context_lemmatized,context_pos,context_normalized,part,lang,positions
0,packen,VVFIN,1806,1,arnim_wunderhorn01_1806-826-9,,"Und als ich kam vor Heidelberg hinan, Da packt...",41:48,0:210,"Und als ich kam vor Heidelberg hinan , Da pack...",9,0:43,"und als ich kommen vor Heidelberg hinan , da p...","KON KOUS PPER VVFIN APPR NE PTKVZ $, ADV VVFIN...","Und als ich kam vor Heidelberg hinan , Da pack...",dwug_de,de,"[41, 48]"
1,packen,VVFIN,1897,1,lasswitz_planeten02_1897-2310-1,,Er packte seine Sachen zusammen und trat aus d...,3:9,0:65,Er packte ſeine Sachen zuſammen und trat aus d...,1,0:13,er packen seine Sache zusammen und treten aus ...,PPER VVFIN PPOSAT NN PTKVZ KON VVFIN APPR ART ...,Er packte seine Sachen zusammen und trat aus d...,dwug_de,de,"[3, 9]"
2,packen,VVFIN,1880,1,werner_seeleben_1880-317-7,,„Der Monat dreht und der Koch packt auf“ laute...,30:35,0:243,„ Der Monat dreht und der Koch packt auf “ lau...,7,0:45,""" d Monat drehen und d Koch packen auf "" laute...",$( ART NN VVFIN KON ART NE VVFIN PTKVZ $( VVFI...,""" Der Monat dreht und der Koch packt auf "" lau...",dwug_de,de,"[30, 35]"


In [3]:
df.part.value_counts()

dwug_de    9125
Name: part, dtype: int64

In [27]:
df.groupby('part').lemma.value_counts().to_csv('part-lemma-counts.tsv', sep='\t')
!column -t part-lemma-counts.tsv

part     lemma               lemma
dwug_de  Abgesang            200
dwug_de  Armenhaus           200
dwug_de  Behandlung          200
dwug_de  Entscheidung        200
dwug_de  Frechheit           200
dwug_de  Fuß                 200
dwug_de  Gesichtsausdruck    200
dwug_de  Knotenpunkt         200
dwug_de  Kubikmeter          200
dwug_de  Lyzeum              200
dwug_de  Mulatte             200
dwug_de  Mut                 200
dwug_de  Rezeption           200
dwug_de  Schmiere            200
dwug_de  Seminar             200
dwug_de  Sensation           200
dwug_de  Tier                200
dwug_de  Titel               200
dwug_de  Truppenteil         200
dwug_de  ausspannen          200
dwug_de  beimischen          200
dwug_de  packen              200
dwug_de  vergönnen           200
dwug_de  voranstellen        200
dwug_de  vorliegen           200
dwug_de  zersetzen           200
dwug_de  überspannen         200
dwug_de  Naturschönheit      198
dwug_de  Engpaß              182
dwug_de 

### sanity check the specified positions of the target words

In [56]:
df['wordat'] = df.apply(lambda r: r.context[slice(*r.positions)], axis=1)
df['wordat_fixed'] = df.apply(lambda r: r.context[slice(*r.positions_fixed)], axis=1)
df['wordat_ctxed'] = df.apply(lambda r: r.context[max(0,r.positions[0]-5):r.positions[1]+5 ], axis=1)
df['wordat_ctxed_fixed'] = df.apply(lambda r: r.context[max(0,r.positions_fixed[0]-5):r.positions_fixed[1]+5 ], axis=1)

In [57]:
mask = df.positions!=df.positions_fixed
print(mask.sum(), mask.mean())
df[mask][['lemma','positions','positions_fixed','wordat','wordat_fixed','wordat_ctxed', 'wordat_ctxed_fixed']].sample(20)

49 0.05875299760191847


,lemma,positions,positions_fixed,wordat,wordat_fixed,wordat_ctxed,wordat_ctxed_fixed
577,друг,"[36, 40]","[0, 6]",доме,Друзья,"его доме, но",Друзья отца
666,человек,"[43, 56]","[26, 30]",человеколюбче,люди,"яжа, человеколюбче.","еси люди, яже"
825,женщина,"[60, 67]","[20, 27]",женщина,женщине,"е, а женщина ждал","ся к женщине, как"
477,день,"[120, 124]","[196, 200]",дѣло,дней,это дѣло спец,"шихъ дней, про"
253,слово,"[71, 76]","[89, 94]",слова,слова,Богослова» ест,акие слова:«Кто
37,тысяча,"[49, 52]","[49, 53]",тыс,тыс.,ло 5 тыс. ваг,ло 5 тыс. ваго
656,жена,"[65, 69]","[33, 38]",жены,женой,обще жены могу,"брой женой, кон"
660,жена,"[148, 152]","[50, 54]",жена,жены,лает жена.,"уход жены; но,"
665,человек,"[184, 189]","[62, 68]",чтобы,людьми,"рам, чтобы они",ными людьми напр
688,человек,"[25, 28]","[169, 174]",что,людей,"том, что осво",ства людей.


In [58]:
df.positions = df.positions_fixed

In [59]:
df.indexes_target_token = df.positions.apply(lambda l: list(map(str, l))).str.join(':')

In [61]:
for l in df.lemma:
    df[df.lemma==l][original_columns].to_csv(f'../RuDSIfixed/data/{l}/uses.csv', sep='\t', index=False)

In [62]:
df.positions.apply(lambda l: list(map(str, l))).str.join(':')

0        10:14
1        20:24
2        27:30
3         9:14
4      346:349
        ...   
835       9:16
836      64:72
837      46:53
838     99:105
839      15:23
Name: positions, Length: 834, dtype: object